In [17]:
#import training data

In [18]:
import json
f = open('train_data.json','r')
train = json.load(f) 
f.close() 

print(len(train["labels"]))

10000


In [19]:
#transform training data word count
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_train_tf=count_vect.fit_transform(train["titles"])
print(X_train_tf.shape)

(10000, 8327)


In [20]:
#apply inverse document frequency - reduced the weight of the words not necessary in categorizing data
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(X_train_tf)
print(X_train_tfidf.shape)

(10000, 8327)


In [21]:
#apply naive bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train["labels"])

In [22]:
#import data to be predict
import json

f = open('tensorflow_opened_issues.json','r')

issues_list = json.load(f) 

print(len(issues_list))

f.close() 

4030


In [23]:
#collect the issues titles, body and labels
data=[]
labels_list=[]
for i in range(len(issues_list)):
    single_data=[]
    issue=issues_list[i]
    issue_title=issue["title"]
    issue_body=issue["body"]
    
    single_data=single_data+[issue_title,issue_body];
    
    data=data+[single_data];
    
    issue_labels_list = issue["labels"]
    single_label_list=[]
    for j in range(len(issue_labels_list)):
        label=issue_labels_list[j]
        label_name=label["name"]
        single_label_list=single_label_list+[label_name]
    labels_list=labels_list+[single_label_list]

In [24]:
print(len(data),len(labels_list))

4030 4030


In [25]:
# collect all the titles of issues
issues_titles=[]
for i in range(len(data)):
    issues_titles=issues_titles+[data[i][0]]
print(len(issues_titles))

4030


In [26]:
#variables to find how many times we are predicting each category
real_cat0=0
real_cat1=0
real_cat2=0
real_cat3=0
real_cat4=0
real_cat5=0
pred_cat0=0
pred_cat1=0
pred_cat2=0
pred_cat3=0
pred_cat4=0
pred_cat5=0
def my_func(a):
    global pred_cat0,pred_cat1,pred_cat2,pred_cat3,pred_cat4,pred_cat5
    if(a==0):
        pred_cat0=pred_cat0+1
    elif(a==1):
        pred_cat1=pred_cat1+1
    elif(a==2):
        pred_cat2=pred_cat2+1
    elif(a==3):
        pred_cat3=pred_cat3+1
    elif(a==4):
        pred_cat4=pred_cat4+1
    elif(a==5):
        pred_cat5=pred_cat5+1



In [27]:
#separate all the issues by their labels or predict labels for them
import re

issue_type=[]
issue_num=[]

predicted_issues=0
non_predicted_issues=0

for i in range(len(labels_list)):
    lb=labels_list[i]
    l=len(lb)
    if(l>0):
        itr=0
        for j in range(len(lb)):
            if(re.search("docs-bug", lb[j])):
                issue_type=issue_type+[['docs-bug']]
                issue_num=issue_num+[0]
                real_cat0=real_cat0+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("bug", lb[j])):
                issue_type=issue_type+[['bug']]
                issue_num=issue_num+[1]
                real_cat1=real_cat1+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("feature", lb[j])):
                issue_type=issue_type+[['feature']]
                issue_num=issue_num+[2]
                real_cat2=real_cat2+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("support", lb[j])):
                issue_type=issue_type+[['support']]
                issue_num=issue_num+[3]
                real_cat3=real_cat3+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("performance", lb[j])):
                issue_type=issue_type+[['performance']]
                issue_num=issue_num+[4]
                real_cat4=real_cat4+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("build/install", lb[j])):
                issue_type=issue_type+[['build/install']]
                issue_num=issue_num+[5]
                real_cat5=real_cat5+1
                non_predicted_issues=non_predicted_issues+1
                break
            itr=itr+1
        if(itr==l):
            #prediction of unrecognized issue
            issue_type=issue_type+[['unrecognized']] #keep it as unrecognized
            
            #transform test data
            X_test_tf=count_vect.transform([issues_titles[i]])
            X_test_tfidf=tfidf_transformer.transform(X_test_tf)
            predicted = clf.predict(X_test_tfidf)
            
            issue_num=issue_num+[predicted[0]]
            predicted_issues=predicted_issues+1
            my_func(predicted[0])
    else:
        #prediction of unrecognized issue
        issue_type=issue_type+[['unrecognized']] #keep it as unrecognized
        
        #transform test data
        X_test_tf=count_vect.transform([issues_titles[i]])
        X_test_tfidf=tfidf_transformer.transform(X_test_tf)
        predicted = clf.predict(X_test_tfidf)
        
        issue_num=issue_num+[predicted[0]]
        predicted_issues=predicted_issues+1
        my_func(predicted[0])
print(len(issue_type))
print(len(issue_num))

4030
4030


In [28]:
#problem in couning
print("no of total issues: ",len(issue_type))
print("no of predicted issues: ",predicted_issues)
print("no of non-predicted issues: ",non_predicted_issues)

no of total issues:  4030
no of predicted issues:  712
no of non-predicted issues:  3318


In [29]:
#maybe its right
print("no of predicted issues:     ",pred_cat0,pred_cat1,pred_cat2,pred_cat3,pred_cat4,pred_cat5)
print("no of non-predicted issues: ",real_cat0,real_cat1,real_cat2,real_cat3,real_cat4,real_cat5)
print("no of predicted issues:     ",pred_cat0+pred_cat1+pred_cat2+pred_cat3+pred_cat4+pred_cat5)
print("no of non-predicted issues: ",real_cat0+real_cat1+real_cat2+real_cat3+real_cat4+real_cat5)

no of predicted issues:      1 480 0 159 0 72
no of non-predicted issues:  173 1387 773 455 158 372
no of predicted issues:      712
no of non-predicted issues:  3318


In [30]:
#issues and their type
for i in range(len(issue_type)):
    print(issue_type[i],issue_num[i])

['build/install'] 5
['unrecognized'] 1
['performance'] 4
['bug'] 1
['build/install'] 5
['unrecognized'] 1
['unrecognized'] 1
['support'] 3
['docs-bug'] 0
['feature'] 2
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['performance'] 4
['support'] 3
['support'] 3
['support'] 3
['bug'] 1
['docs-bug'] 0
['unrecognized'] 3
['support'] 3
['support'] 3
['bug'] 1
['support'] 3
['build/install'] 5
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['support'] 3
['unrecognized'] 3
['bug'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['support'] 3
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 5
['performance'] 4
['unrecognized'] 1
['support'] 3
['unrecognized'] 1
['support'] 3
['support'] 3
['unrecognized'] 1
['bug'] 1

['bug'] 1
['bug'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['unrecognized'] 1
['feature'] 2
['build/install'] 5
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['performance'] 4
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['bug'] 1
['unrecognized'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['feature'] 2
['support'] 3
['bug'] 1
['docs-bug'] 0
['support'] 3
['feature'] 2
['bug'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['bug'] 1
['feature'] 2
['bug'] 1
['feature'] 2
['performance'] 4
['build/install'] 5
['bug'] 1
['bug'] 1
['support'] 3
['unrecognized'] 1
['feature'] 2
['unrecognized'] 3
['bug'] 1
['bug'] 1
['support'] 3
['unrecognized'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5
['unrecognized'] 1


['unrecognized'] 3
['unrecognized'] 3
['unrecognized'] 3
['unrecognized'] 3
['feature'] 2
['build/install'] 5
['support'] 3
['feature'] 2
['unrecognized'] 3
['bug'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['bug'] 1
['support'] 3
['performance'] 4
['feature'] 2
['build/install'] 5
['build/install'] 5
['support'] 3
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2
['build/install'] 5
['build/install'] 5
['feature'] 2
['bug'] 1
['bug'] 1
['build/install'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2
['performance'] 4
['feature'] 2
['bug'] 1
['bug'] 1
['feature'] 2
['build/install'] 5
['bug'] 1
['feature'] 2
['bug'] 1
['docs-bug'] 0
['support'] 3
['bug'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['feature'] 2
['unrecognized'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['bug'] 1
['support'] 3
['support'] 3
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2


['bug'] 1
['bug'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['performance'] 4
['feature'] 2
['bug'] 1
['bug'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['feature'] 2
['bug'] 1
['build/install'] 5
['feature'] 2
['feature'] 2
['unrecognized'] 3
['feature'] 2
['performance'] 4
['bug'] 1
['feature'] 2
['feature'] 2
['performance'] 4
['docs-bug'] 0
['support'] 3
['performance'] 4
['performance'] 4
['support'] 3
['support'] 3
['unrecognized'] 1
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['unrecognized'] 1
['feature'] 2
['feature'] 2
['bug'] 1
['performance'] 4
['build/install'] 5
['docs-bug'] 0
['bug'] 1
['support'] 3
['bug'] 1
['feature'] 2
['performance'] 4
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['support'] 3
['build/install'] 5
['build/install'] 5
['feature'] 2
['p

['docs-bug'] 0
['feature'] 2
['unrecognized'] 1
['docs-bug'] 0
['support'] 3
['feature'] 2
['docs-bug'] 0
['feature'] 2
['docs-bug'] 0
['support'] 3
['support'] 3
['docs-bug'] 0
['bug'] 1
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['bug'] 1
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['bug'] 1
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['docs-bug'] 0
['support'] 3
['support'] 3
['docs-bug'] 0
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['docs-bug'] 0
['docs-bug'] 0
['build/install'] 5
['feature'] 2
['feature'] 2
['docs-bug'] 0
['feature'] 2
['bug'] 1
['feature'] 2
['build/install'] 5
['feature'] 2
['bug'] 1
['support'] 3
['support'] 3
['support'] 3
['build/install'] 5
['bug'] 1
['feature'] 2
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['docs-bug'] 0
['bug'] 1
['bug'] 1
['support'] 3
['unrecognized'] 5
['bug'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['support'] 3
['bug'] 

['unrecognized'] 3
['unrecognized'] 1
['feature'] 2
['build/install'] 5
['unrecognized'] 3
['unrecognized'] 1
['unrecognized'] 1
['feature'] 2
['feature'] 2
['unrecognized'] 1
['bug'] 1
['support'] 3
['unrecognized'] 3
['unrecognized'] 3
['feature'] 2
['unrecognized'] 1
['unrecognized'] 3
['feature'] 2
['unrecognized'] 1
['unrecognized'] 3
['feature'] 2
['unrecognized'] 3
['support'] 3
['support'] 3
['support'] 3
['support'] 3
['unrecognized'] 5
['feature'] 2
['support'] 3
['unrecognized'] 5
['feature'] 2
['support'] 3
['support'] 3
['bug'] 1
['feature'] 2
['bug'] 1
['feature'] 2
['unrecognized'] 3
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['support'] 3
['bug'] 1
['feature'] 2
['feature'] 2
['feature'] 2
['support'] 3
['feature'] 2
['feature'] 2
['feature'] 2
['build/install'] 5
['bug'] 1
['feature'] 2
['feature'] 2
['support'] 3
['support'] 3
['support'] 3
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['feature

In [31]:
# total number of issues in each category
list1=issue_num
cat0=0
cat1=0
cat2=0
cat3=0
cat4=0
cat5=0
for i in range(len(list1)):
    if(list1[i]==0):
        cat0=cat0+1
    elif(list1[i]==1):
        cat1=cat1+1
    elif(list1[i]==2):
        cat2=cat2+1
    elif(list1[i]==3):
        cat3=cat3+1
    elif(list1[i]==4):
        cat4=cat4+1
    elif(list1[i]==5):
        cat5=cat5+1
print(cat0,cat1,cat2,cat3,cat4,cat5)

174 1867 773 614 158 444


In [32]:
first_issue=issues_list[0]
print(first_issue)

{'url': 'https://api.github.com/repos/tensorflow/tensorflow/issues/47505', 'repository_url': 'https://api.github.com/repos/tensorflow/tensorflow', 'labels_url': 'https://api.github.com/repos/tensorflow/tensorflow/issues/47505/labels{/name}', 'comments_url': 'https://api.github.com/repos/tensorflow/tensorflow/issues/47505/comments', 'events_url': 'https://api.github.com/repos/tensorflow/tensorflow/issues/47505/events', 'html_url': 'https://github.com/tensorflow/tensorflow/issues/47505', 'id': 820060615, 'node_id': 'MDU6SXNzdWU4MjAwNjA2MTU=', 'number': 47505, 'title': 'install_pip_packages.sh specifies incompatible version combo for pylint and astroid', 'user': {'login': 'deven-amd', 'id': 36858332, 'node_id': 'MDQ6VXNlcjM2ODU4MzMy', 'avatar_url': 'https://avatars.githubusercontent.com/u/36858332?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/deven-amd', 'html_url': 'https://github.com/deven-amd', 'followers_url': 'https://api.github.com/users/deven-amd/followers', 'follow

In [33]:
#prioritize issues based on amount of conversation
number_of_comments=[]
isassigned=[]
for i in range(len(issues_list)):
    print(issues_list[i]["comments"])

1
0
2
1
1
1
0
2
1
0
1
0
0
0
2
1
1
0
0
0
1
2
4
0
3
1
7
2
1
2
1
2
2
1
9
2
1
4
1
2
2
1
20
6
4
4
1
1
3
4
2
1
1
5
0
1
4
2
0
1
4
1
3
2
4
3
4
0
2
1
3
2
2
0
3
3
3
4
1
2
5
3
0
1
0
1
2
3
0
4
2
11
1
2
4
1
0
3
5
1
2
0
3
3
1
2
4
1
7
0
5
0
3
1
11
0
1
2
10
1
2
4
2
1
2
2
6
8
4
2
2
5
3
1
2
4
0
0
1
0
0
0
2
4
3
1
7
5
2
1
1
0
1
1
7
4
10
2
2
3
4
2
2
6
10
4
4
1
5
2
0
2
1
2
4
4
12
0
2
6
8
1
2
2
3
2
2
5
3
3
11
2
14
7
4
1
2
1
3
0
2
3
1
4
2
11
4
4
2
1
1
5
0
10
6
3
1
0
5
7
7
6
2
5
0
7
3
6
1
4
7
5
8
0
3
2
0
6
2
1
2
1
3
5
2
0
3
0
0
6
7
5
3
6
3
1
1
0
0
1
0
0
1
2
6
6
7
2
3
4
4
0
1
4
1
5
1
1
1
11
0
0
1
0
1
3
21
0
7
8
11
2
2
1
3
0
10
2
4
1
1
1
2
1
2
1
1
1
3
1
1
2
1
1
4
4
2
1
2
1
1
2
2
1
1
1
5
1
3
6
10
3
5
4
3
5
6
1
6
0
2
8
1
2
5
1
4
1
1
0
0
9
5
11
2
2
4
2
2
8
5
3
0
3
7
7
0
0
0
2
0
2
5
4
2
13
3
4
7
0
8
0
1
1
3
2
4
0
0
4
2
2
2
3
3
0
1
2
5
1
2
4
0
4
0
0
15
0
0
9
10
2
0
3
6
8
1
3
0
5
1
3
4
13
3
1
3
3
1
4
3
1
2
4
2
2
2
6
0
2
3
9
0
4
1
1
1
1
20
4
1
9
6
5
3
0
4
9
8
0
1
2
4
3
7
1
8
3
3
0
2
2
3
2
17
2
1
2
6
6
5
3
5
2
0
2
3
0
6

In [34]:
#prioritize issues based on the autor_association
number_of_comments=[]
isassigned=[]
for i in range(len(issues_list)):
    print(issues_list[i]["author_association"])

CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
MEMBER
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
MEMBER
NONE
MEMBER
NONE
CONTRIBUTOR
MEMBER
NONE
MEMBER
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
MEMBER
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
CONTRIBUTOR
CONTRIBUTOR
MEMBER
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR


NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
NONE
NONE
MEMBER
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRI

CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
MEMBER
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
MEMBER
MEMBER
MEMBER
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
NONE
NONE
CONTRIBUTOR
NONE
NONE
CONTRIBUTOR
CONTRIBUTOR
CONTRIBUTOR
NONE
NONE
NONE
NONE
NONE
NON

In [39]:
#prioritize issues based on the assignees
number_of_comments=[]
isassigned=[]
for i in range(len(issues_list)):
    if(issues_list[i]["assignee"]):
        print(issues_list[i]["assignee"]["type"])
    else:
        print("none")
    

User
User
User
User
User
none
none
User
User
User
User
none
none
none
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User


User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
none
User
User
User
User
User
User
User
User
User
User
User
User
User
none
User
User
User
User
User
User
User
none
User
User
User
User
User
User
User
User
User
User
User
User
none
User
User
none
User
User
User
User
User
User
User
none
User
User
User
User
User
User
User
User
User
User
User
none
User
User
User
User
none
User
User
User
none
User
none
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
User
none
User
none
User
User
User
none
User
User
User
User
User
none
User
User
User
User
User
User
User
User
User
none
User
none
none
none
User
User
none
User
User
User
User
User
User
User
User
User
User
User
User
none
User
User
User
User
User
User
User
User
User
User
User
User
User
User
none
none
User
User
none
User
User
User
User
User
User
User
User
User
User
User
none
User
User
User
User
User
User
User


In [40]:
#prioritize issues based on the milestones
number_of_comments=[]
isassigned=[]
for i in range(len(issues_list)):
    if(issues_list[i]["milestone"]):
        print(issues_list[i]["milestone"])
    else:
        print("none")

none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none


none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none


In [43]:
#prioritize issues based on the milestones
locked=0;
for i in range(len(issues_list)):
    if(issues_list[i]["locked"]):
        locked=locked+1
print(locked)

2
